<a href="https://colab.research.google.com/github/JuflN/Projeto-inteligencia-artificial-2025.3/blob/main/Inteligencia_Artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

### **PROJETO PRÁTICO** [LangChain + Grandes Modelos de Linguagem]


O **PROJETO PRÁTICO** deve ser feito utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook armazenado no Google Drive e o link de um repositório no GitHub devem ser enviados usando o seguinte formulário:

> https://forms.gle/D4gLqP1iGgyn2hbH8


**IMPORTANTE**: A submissão deve ser feita até o dia **07/12 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`JENIFER SOARES SOUZA           11202020219`

**Integrante 02:**

`JUAN PABLO MORENO OLIVEIRA     11202131494`
**Integrante 01:**

`Raul Henrique Leal de Araújo   11202111573`

**Integrante 02:**

`JUAN PABLO MORENO OLIVEIRA     11202131494`
**Integrante 01:**

`JENIFER SOARES SOUZA           11202020219`


### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

Dados do LLM selecionada:

>


**LLM**: GPT-4.1 mini ou gemini-2.5-flash

>

**Link para a documentação oficial**: https://openai.com/index/gpt-4-1/ ou https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash?hl=pt-br



### **API**
---

Dados da API selecionada:

**API**:Gutendex

**Site oficial**:https://gutendex.com/

**Link para a documentação oficial**:https://gutendex.com/ (na homepage)






### **DESCRIÇÃO**
---

O projeto propõe o desenvolvimento de uma aplicação de `Aprendizado de
Máquina` (AM) e `Processamento de Linguagem Natural` (PLN) com o objetivo de
analisar e classificar personagens de obras literárias de acordo com seus traços de
personalidade, inferidos a partir de trechos selecionados dos livros.
Como prova de conceito, será utilizada a série “Os Karas”, de Pedro
Bandeira, com o intuito de construir um modelo que identifique padrões linguísticos,
emocionais e comportamentais que possam descrever o perfil de cada personagem.
A aplicação integra a etapa de classificação com uma LLM (Large Language
Model) acessada via API educacional, responsável por expandir a análise, sugerir
exemplos semelhantes e propor novos perfis de personagens com base em
entradas inéditas fornecidas pelo usuário.




### **IMPLEMENTAÇÃO**
---

Neste trabalho, utilizamos como fonte de dados o [Projeto Gutenberg](https://www.gutenberg.org/about/), que disponibiliza obras literárias em domínio público. A partir desse acervo, selecionamos quatro livros brasileiros de interesse para análise — *Dom Casmurro*, *O Cortiço* e *Quincas Borba*. Para integrá-los ao pipeline, realizamos o mapeamento manual de seus respectivos *IDs* no catálogo e utilizamos a API [Gutendex](https://gutendex.com/) para recuperar metadados essenciais, incluindo as URLs que fornecem o texto completo em formato **.txt**.

#### Instalação de bibliotecas

In [ ]:
!pip install -U "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      S

#### Importação de bibliotecas

In [ ]:
import csv, nltk, spacy, json, os, joblib, re, requests, unicodedata, ast, pandas as pd, numpy as np
from google.colab import userdata
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, HttpUrl
from spacy.lang.en.stop_words import STOP_WORDS as EN_STOPWORDS
from time import sleep
from typing import List, Optional, Dict, Set
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, hamming_loss, f1_score, precision_recall_curve
from sklearn.calibration import CalibratedClassifierCV
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### Definição dos modelos de dados

In [ ]:
class Person(BaseModel):
    name: str
    birth_year: Optional[int] = None
    death_year: Optional[int] = None


class BookMetadataInput(BaseModel):
    id: int
    title: str
    authors: List[Person]
    summaries: List[str] = []
    editors: List[Person] = []
    translators: List[Person] = []
    subjects: List[str] = []
    bookshelves: List[str] = []
    languages: List[str] = []
    copyright: Optional[bool] = None
    media_type: Optional[str] = None
    formats: Dict[str, HttpUrl]
    download_count: int


class BookMetadataOutput(BaseModel):
    title: str
    authors: List[Person]
    summaries: List[str] = []
    subjects: List[str] = []
    characters: List[str] = []
    languages: List[str] = []
    copyright: Optional[bool] = None
    media_type: Optional[str] = None
    text_url: HttpUrl


class CharacterLLMInput(BaseModel):
    book: BookMetadataOutput
    character_name: str
    snippets: Dict[int, str]

#### Definição de funções auxiliares

In [ ]:
def _normalize_name(name: str) -> str:
    name = name.strip().strip('"').strip("'").strip()
    name = re.sub(r"[’'´`]+s$", "", name)
    return name

def _normalize_text(texto: str) -> str:
    if not isinstance(texto, str):
        return ""
    stopwords_pt = set(stopwords.words("portuguese"))
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8", "ignore")
    texto = texto.lower()
    texto = re.sub(r"http\S+|www\S+", " ", texto)
    texto = re.sub(r"\d+", " ", texto)
    texto = re.sub(r"[^\w\s]", " ", texto)
    texto = " ".join([t for t in texto.split() if t not in stopwords_pt])
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto


def _name_tokens(name: str) -> Set[str]:
    return {t for t in re.split(r"\s+", name) if t}


def _author_token_set(authors) -> Set[str]:
    tokens = set()
    for a in authors:
        norm = _normalize_name(a.name.replace(",", " "))
        tokens.update(_name_tokens(norm))
    return tokens


def _is_stop_candidate(token_text: str) -> bool:
    if not token_text:
        return True
    if token_text.lower() in EN_STOPWORDS:
        return True
    if len(token_text) <= 2:
        return True
    return False


def _strip_accents(s: str) -> str:
    nfkd = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in nfkd if unicodedata.category(ch) != "Mn")


def _split_paragraphs(text: str) -> List[str]:
    raw_paragraphs = re.split(r"\n\s*\n", text)
    return [p.strip() for p in raw_paragraphs if p.strip()]


def extract_book_metadata(book: int) -> BookMetadataInput:
    response = requests.get(f"https://gutendex.com/books/{book}")
    response.raise_for_status()

    gutendex = BookMetadataInput.model_validate(response.json())
    return gutendex


def extract_character_names(book) -> List[str]:
    text = " ".join(book.summaries or [])
    doc = nlp(text)

    author_tokens = _author_token_set(book.authors)
    raw_candidates: Set[str] = set()

    for ent in doc.ents:
        if ent.label_ != "PERSON":
            continue

        name = _normalize_name(ent.text)
        if not name:
            continue

        tokens = _name_tokens(name)
        if not tokens:
            continue

        if tokens.issubset(author_tokens):
            continue

        raw_candidates.add(name)

    return sorted(raw_candidates)


def enrich_books_with_characters(book: BookMetadataInput) -> BookMetadataOutput:
    return BookMetadataOutput(
        authors=book.authors,
        title=book.title,
        summaries=book.summaries,
        subjects=book.subjects,
        characters=extract_character_names(book),
        text_url=book.formats['text/plain; charset=us-ascii']
    )


def find_relevant_paragraphs(
        paragraphs: List[str],
        character_name: str,
        window: int = 1,
        min_length: int = 80,
) -> List[str]:
    max_snippets_in_prompt = 20

    name_norm = _strip_accents(character_name).lower()
    pattern = re.compile(rf"\b{re.escape(name_norm)}\b")

    hit_indices: List[int] = []
    for i, p in enumerate(paragraphs):
        p_norm = _strip_accents(p).lower()
        if pattern.search(p_norm):
            hit_indices.append(i)

    if not hit_indices:
        return {}

    selected_indices: List[int] = []
    seen: Set[int] = set()

    for idx in hit_indices:
        start = max(0, idx - window)
        end = min(len(paragraphs), idx + window + 1)

        for j in range(start, end):
            if j not in seen:
                seen.add(j)
                selected_indices.append(j)

    selected_indices.sort()
    return {
        i: paragraphs[i].replace("\r\n", "\n").replace("\r", "")
        for i in selected_indices[:max_snippets_in_prompt] if len(paragraphs[i].strip()) >= min_length
    }


def build_character_llm_input(
        book_output: BookMetadataOutput,
        character_name: str,
        window: int = 1
) -> CharacterLLMInput:
    response = requests.get(book_output.text_url)
    response.raise_for_status()

    paragraphs = _split_paragraphs(response.text)
    snippets = find_relevant_paragraphs(
        paragraphs,
        character_name=character_name,
        window=window
    )

    return CharacterLLMInput(
        book=book_output,
        character_name=character_name,
        snippets=snippets
    )


def build_prompt_from_context(ctx: CharacterLLMInput) -> str:
    data = json.dumps({
        "metadados_livro": {
            "titulo": ctx.book.title,
            "autores": [a.model_dump() for a in ctx.book.authors],
            "assuntos": ctx.book.subjects,
            "personagens_detectados_no_resumo": ctx.book.characters
        },
        "personagem_alvo": {
            "nome": ctx.character_name
        },
        "trechos_texto": ctx.snippets
    }, ensure_ascii=False)
    return data


def process_book_characters(book: BookMetadataOutput) -> List[dict]:
    results: List[dict] = []

    for character_name in book.characters:
        try:
            if not character_name.strip:
                continue

            print(f"\n\nProcessando {character_name} para o livro {book.title}")
            ctx = build_character_llm_input(
                book_output=book,
                character_name=character_name,
                window=1,
            )

            if not book.characters:
                print(f"Nenhum personagem encontrado para o livro {book.title}")
                continue

            if not ctx.snippets:
                print(f"Nenhum parágrafo encontrado personagem de nome {character_name} encontrado para o livro {book.title}")

            prompt_text = build_prompt_from_context(ctx)
            llm_output = trait_chain.invoke({"texto": prompt_text})

            results.append(
                {
                    "character_name": character_name,
                    "character_context": ctx,
                    "llm_output": llm_output,
                }
            )

            build_row_from_result(ctx, llm_output)
            sleep(1)

        except Exception as error:
            print("Falha ao extrair dados do personagem", str(error))

    return results


def build_row_from_result(ctx: CharacterLLMInput, llm_output: str) -> List[dict]:
    book = ctx.book

    authors_json = json.dumps([a.model_dump() for a in book.authors], ensure_ascii=False)
    summaries_json = json.dumps(book.summaries, ensure_ascii=False)
    subjects_json = json.dumps(book.subjects, ensure_ascii=False)
    characters_json = json.dumps(book.characters, ensure_ascii=False)

    cleaned = re.sub(r"^```(\w+)?", "", llm_output.strip())
    cleaned = re.sub(r"```$", "", cleaned).strip()

    inferences = json.loads(cleaned)["inferences"]
    resume = json.loads(cleaned)["resume"]

    results = []
    for inference in inferences:
        row = {
            "title": book.title,
            "authors": authors_json,
            "summaries": summaries_json,
            "subjects": subjects_json,
            "characters": characters_json,
            "text_url": book.text_url,
            "character_name": ctx.character_name,
            "character_resume": resume,
            "snippet": ctx.snippets.get(int(inference["id"])),
            "traits": json.dumps(inference["traits"], ensure_ascii=False),
        }

        append_row_to_csv(row)
        results.append(row)

    return results


def append_row_to_csv(row: dict, path: str = "characters_dataset.csv") -> None:
    file_exists = os.path.exists(path)

    with open(path, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

#### Configuração para a pipeline



In [ ]:
nlp = spacy.load("en_core_web_sm")

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
print(f"\n\nGOOGLE_API_KEY está vazia." if GOOGLE_API_KEY is None else "")

llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.0,
)

trait_prompt = PromptTemplate(
    input_variables=["texto"],
    template=("""
        Você é um analista especializado em perfis de personagens literários. Com base EXCLUSIVAMENTE nos trechos fornecidos abaixo, produza um JSON com inferências de personalidade. Regras obrigatórias:
        - A saída deve conter apenas JSON. Não inclua explicações.
        - O JSON final deve ter exatamente as chaves:
        - "inferences": lista de objetos.
        - "resume": um resumo com NO MÁXIMO 200 caracteres EM PORTUGUÊS BARSILEIRO sobre quem é o personagem com base EM TODOS OS TRECHOS enviado no contexto como "trechos_texto".
        - Cada objeto dentro de "inferences" deve seguir esta estrutura:
        - "id": o mesmo id recebido na entrada, correspondente ao trecho analisado.
        - "traits": lista de adjetivos que descrevem a personalidade detectada NO TRECHO.
        - Você deve retirar apenas traits claramente justificáveis pelo trecho.
        - Se não houver traits relevantes, NÃO inclua esse item na lista "inferences".
        - Todos os adjetivos devem ser:
        - em PORTUGUÊS BRASILEIRO,
        - no gênero masculino,
        - escolhidos APENAS desta lista:
         "astuto","inteligente","misterioso","impulsivo","determinado",
          "corajoso","leal","observador","protetor","curioso",
          "arrogante","orgulhoso","cínico","vaidoso","ambicioso",
          "teimoso","dominador","controlador","inflexível","desconfiado",
          "gentil","bondoso","generoso","altruísta","empático",
          "frio","calculista","rígido","prudente","precavido",
          "racional","sarcástico","irônico","questionador","criativo",
          "trabalhador","resiliente","responsável","conservador","pragmático",
          "agressivo","hostil","vingativo","cruel","mentiroso",
          "covarde","irresponsável","negligente","egoísta","maldoso",
        - Contexto: \n {texto}
    """),
)

trait_chain = trait_prompt | llm_model | StrOutputParser()

#### Definição da pipeline

In [ ]:
fetch_metadata_runnable = RunnableLambda(
    lambda inp: extract_book_metadata(inp["book_id"])
)

enrich_with_characters_runnable = RunnableLambda(
    lambda book: enrich_books_with_characters(book)
)

book_metadata_pipeline = fetch_metadata_runnable | enrich_with_characters_runnable
full_book_pipeline = book_metadata_pipeline | RunnableLambda(process_book_characters)

#### Execução da pipeline

In [ ]:
if __name__ == "__main__":

    path = 'final_ptbr_fiction.csv'
    if os.path.exists(path):
        df = pd.read_csv(path)
        book_ids = df['id']
    else:
        print("Arquivo não encontrado, usando um ID de teste.")
        book_ids = [69187]

    for book_id in book_ids:
        try:
            pipeline_input = {"book_id": book_id}
            results = full_book_pipeline.invoke(pipeline_input)

            for index, item in enumerate(results):
                print(f"\n\nResultado {index + 1}:")
                print(item)

                sleep(5)
        except Exception as error:
            print("Falha ao executar extração de dados para o livro", str(error))



Processando Graziella para o livro A capital federal (impressões de um sertanejo)


Processando Sertanejo para o livro A capital federal (impressões de um sertanejo)


Resultado 1:
{'character_name': 'Graziella', 'character_context': CharacterLLMInput(book=BookMetadataOutput(title='A capital federal (impressões de um sertanejo)', authors=[Person(name='Coelho Netto, Henrique', birth_year=1864, death_year=1934)], summaries=['"A Capital Federal (Impressões de um Sertanejo)" by Henrique Coelho Netto is a novel written in the early 20th century. The work likely explores the contrasts between rural life and urban experiences in Brazil, highlighting the thoughts and reflections of the main character, Anselmo, as he transitions from the countryside to the busy city. The narrative promises insights into the socio-political landscape of the time, framed through Anselmo\'s personal journey and encounters.  The opening of the novel introduces us to Anselmo, who is traveling to the city of Rio de

Feedback: block_reason: PROHIBITED_CONTENT



Falha ao extrair dados do personagem Expecting value: line 1 column 1 (char 0)


Processando Luiz Garcia para o livro Yayá Garcia


Feedback: block_reason: PROHIBITED_CONTENT



Falha ao extrair dados do personagem Expecting value: line 1 column 1 (char 0)


Processando Raymundo para o livro Yayá Garcia


Processando Yayá para o livro Yayá Garcia


Processando Yayá Garcia para o livro Yayá Garcia


Resultado 1:
{'character_name': 'Luiz', 'character_context': CharacterLLMInput(book=BookMetadataOutput(title='Yayá Garcia', authors=[Person(name='Machado de Assis', birth_year=1839, death_year=1908)], summaries=['"Yayá Garcia" by Machado de Assis is a novel written in the early 20th century. The book revolves around the life of Luiz Garcia, a public servant living a solitary and routine existence in Santa Thereza, and his affectionate relationship with his young daughter, Yayá, as well as a family servant named Raymundo. It delves into themes of love, duty, and the complexities of human relationships against the backdrop of 19th century Brazilian society.  The opening of "Yayá Garcia" introduces us to Luiz Garcia, a widower who has chosen a life of seclusion and rou

In [ ]:
from google.colab import files

# Especifica o arquivo que deseja baixar
files.download('characters_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pd.read_csv("characters_dataset.csv").head(20)


,title,authors,summaries,subjects,characters,text_url,character_name,character_resume,snippet,traits
0,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Graziella,"Graziella é uma professora italiana, inicialme...","O conservador, com um gesto nobre, rejeitou; e...","[""misterioso"", ""gentil"", ""inteligente"", ""pragm..."
1,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Sertanejo,"Sertanejo é um homem vaidoso, orgulhoso e arro...","--Sim, meu tio; tenho aqui. Accendi o charuto ...","[""conservador""]"
2,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Sertanejo,"Sertanejo é um homem vaidoso, orgulhoso e arro...",O Rio começava a apparecer-me. A victoria corr...,"[""vaidoso"", ""orgulhoso"", ""arrogante"", ""dominad..."
3,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Sertanejo,"Sertanejo é um homem vaidoso, orgulhoso e arro...","A victoria parou. Saltámos e eu, curioso de vê...","[""curioso"", ""observador""]"
4,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Sertanejo,"Sertanejo é um homem vaidoso, orgulhoso e arro...",Os rumores deliciavam-me e enterneciam-me--dei...,"[""observador""]"
5,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Sertanejo,"Sertanejo é um homem vaidoso, orgulhoso e arro...","Leve, longinquo, soava o sino da parochia, ora...","[""curioso"", ""corajoso""]"
6,A capital federal (impressões de um sertanejo),"[{""name"": ""Coelho Netto, Henrique"", ""birth_yea...","[""\""A Capital Federal (Impressões de um Sertan...","[""Brazilian literature"", ""Romance fiction, Bra...","[""Graziella"", ""Sertanejo""]",https://www.gutenberg.org/ebooks/70121.txt.utf-8,Sertanejo,"Sertanejo é um homem vaidoso, orgulhoso e arro...",Que trazia eu que me demonstrasse ter vivido n...,"[""questionador"", ""determinado"", ""pragmático"", ..."
7,Flirts,"[{""name"": ""Vasconcellos, Henrique de"", ""birth_...","[""\""Flirts\"" by Henrique de Vasconcellos is a ...","[""Fiction""]","[""Maria do Carmo""]",https://www.gutenberg.org/ebooks/35073.txt.utf-8,Maria do Carmo,"Maria do Carmo é uma mulher de trinta anos, ca...","Maria do Carmo, curvada sobre a meza, folheia ...","[""arrogante""]"
8,Flirts,"[{""name"": ""Vasconcellos, Henrique de"", ""birth_...","[""\""Flirts\"" by Henrique de Vasconcellos is a ...","[""Fiction""]","[""Maria do Carmo""]",https://www.gutenberg.org/ebooks/35073.txt.utf-8,Maria do Carmo,"Maria do Carmo é uma mulher de trinta anos, ca...","MARIA DO CARMO--trinta annos, com dez de casam...","[""curioso"", ""misterioso"", ""maldoso""]"
9,"A confissão de Lucio,: Narrat

In [ ]:
df = pd.read_csv('characters_dataset.csv')
print(df.shape)

(1536, 10)


geramos um csv com 1546 itens, que usaremos para treinar um modelo de classificação com base nos personality traits que definimos. Seria interessante identificar se temos todas as possibilidades de traits.

In [ ]:
# criando a pasta
os.makedirs("/content/data", exist_ok=True)

TRAITS_CSV_PATH = "characters_dataset.csv"
MODEL_PATH = "data/traits_pipeline.joblib"
LABELS_PATH = "data/traits_labels.json"
THRESH_PATH = "data/traits_thresholds.npy"

In [ ]:
def parse_traits_cell(cell):
    if cell is None or str(cell).strip() == "":
        return []
    s = str(cell).strip()
    try:
        parsed = ast.literal_eval(s)
        if isinstance(parsed, (list,tuple)):
            return [str(x).strip() for x in parsed if str(x).strip()]
    except Exception:
        for sep in [";", ",", "|", "/"]:
            if sep in s:
                parts = [p.strip() for p in s.split(sep) if p.strip()]
                if parts:
                    return parts
    return [s]

df['trait_list'] = df['traits'].apply(parse_traits_cell)
df = df[df["snippet"].str.strip() != ""].copy()
print("Linhas após remover snippets vazios:", len(df))
display(df.head(20)['traits'])



Linhas após remover snippets vazios: 1536


,traits
0,"[""misterioso"", ""gentil"", ""inteligente"", ""pragm..."
1,"[""conservador""]"
2,"[""vaidoso"", ""orgulhoso"", ""arrogante"", ""dominad..."
3,"[""curioso"", ""observador""]"
4,"[""observador""]"
5,"[""curioso"", ""corajoso""]"
6,"[""questionador"", ""determinado"", ""pragmático"", ..."
7,"[""arrogante""]"
8,"[""curioso"", ""misterioso"", ""maldoso""]"
9,"[""misterioso""]"


Agora que temos um conjunto de dados, podemos buscar alguns dos personagens da literatura e verificar os traits de acordo com suas histórias. Vamos dar uma olhada nos traços de personalidade de João Romão, o proprietário do cortiço, obra de Eça de Queiroz e um grande clássico da literatura.

In [ ]:
def show_examples(name, n=3):
  sub = df[df['character_name'].str.contains(name, case = False, na = False )]
  display(sub[["character_name","snippet","traits"]].head(n))

show_examples("João Romão")

,character_name,snippet,traits
831,João Romão,"João Romão foi, dos treze aos vinte e cinco an...","[""trabalhador"", ""prudente""]"
832,João Romão,"Proprietario e estabelecido por sua conta, o r...","[""ambicioso"", ""determinado"", ""trabalhador"", ""r..."
833,João Romão,João Romão mostrou grande interesse por esta d...,"[""astuto"", ""calculista""]"


Agora para podermos fazer bom uso de todos esses dados, uma forma mais criativa e **econômica** de categorizar corpus, agora que temos um bom dataset, seria um modelo de classificação. vamos usar nossos dados para treino e teste de um modelo e após isso, fazer uma prova de conceito.

In [ ]:
model_df = df[["snippet","trait_list"]]
model_df.head()

,snippet,traits
0,"O conservador, com um gesto nobre, rejeitou; e...","[""misterioso"", ""gentil"", ""inteligente"", ""pragm..."
1,"--Sim, meu tio; tenho aqui. Accendi o charuto ...","[""conservador""]"
2,O Rio começava a apparecer-me. A victoria corr...,"[""vaidoso"", ""orgulhoso"", ""arrogante"", ""dominad..."
3,"A victoria parou. Saltámos e eu, curioso de vê...","[""curioso"", ""observador""]"
4,Os rumores deliciavam-me e enterneciam-me--dei...,"[""observador""]"


# Treinamento do Modelo

Para treinarmos nosso modelo, Definimos um limite de 50

In [ ]:
TRAITS_FULL = [
    "astuto","inteligente","misterioso","impulsivo","determinado",
    "corajoso","leal","observador","protetor","curioso",
    "arrogante","orgulhoso","cínico","vaidoso","ambicioso",
    "teimoso","dominador","controlador","inflexível","desconfiado",
    "gentil","bondoso","generoso","altruísta","empático",
    "frio","calculista","rígido","prudente","precavido",
    "racional","sarcástico","irônico","questionador","criativo",
    "trabalhador","resiliente","responsável","conservador","pragmático",
    "agressivo","hostil","vingativo","cruel","mentiroso",
    "covarde","irresponsável","negligente","egoísta","maldoso"
]

df_extra = pd.read_csv('traits_dataset_50x200.csv')
#df_extra = df_extra.rename(columns={"trait": "traits"}) Se eu errei, Checar
# Concatenar
df_final = pd.concat([df_extra, model_df], ignore_index=True)

if "text_for_model" in df_final.columns and "snippet" not in df_final.columns:
    df_final = df_final.rename(columns={"text_for_model": "snippet"})
if "trait" in df_final.columns and "traits" not in df_final.columns:
    df_final = df_final.rename(columns={"trait": "traits"})

df_final = df_final.dropna(subset=["snippet", "traits"]).copy()
df_final["snippet"] = df_final["snippet"].astype(str).str.strip()

df_final["trait_list"] = df_final["traits"].apply(parse_traits_cell)


In [ ]:
df_final.head()

,snippet,traits,trait_list
0,A person marked by astuto moved through the sc...,astuto,[astuto]
1,A person marked by astuto moved through the sc...,astuto,[astuto]
2,A person marked by astuto moved through the sc...,astuto,[astuto]
3,People often described them as astuto and it s...,astuto,[astuto]
4,"Known for being astuto, this figure did not he...",astuto,[astuto]


In [1]:
df_final["snippet_clean"] = model_df["snippet"].astype(str).apply(_normalize_text)

NameError: name 'model_df' is not defined

In [ ]:
base = LogisticRegression(max_iter=2000)

calibrated_lr = CalibratedClassifierCV(
    estimator=base,
    method="sigmoid",
    cv=2
)

clf = OneVsRestClassifier(calibrated_lr, n_jobs=-1)


X_texts = df_final["snippet_clean"].astype(str).tolist()
Y_list = df_final["trait_list"].tolist()

print("len(X_texts2):", len(X_texts))
print("len(Y_list2):", len(Y_list))

use_classes = False
mlb = MultiLabelBinarizer(classes=TRAITS_FULL)

Y = mlb.fit_transform(Y_list)

print("Y.shape:", Y.shape)
print("Labels (n):", len(mlb.classes_))

assert len(X_texts) == Y.shape[0], f"Inconsistência de tamanhos: len(X_texts2)={len(X_texts)} vs Y2.shape[0]={Y.shape[0]}"
print("Shapes consistentes. Y2.shape:", Y2.shape)


X_train, X_test, Y_train, Y_test = train_test_split(
    X_texts, Y, test_size=0.20, random_state=42, shuffle=True
)

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=30000, ngram_range=(1,2), min_df=2)),
    ("clf", OneVsRestClassifier(
                CalibratedClassifierCV(
                    estimator=LogisticRegression(max_iter=2000),
                    method="sigmoid",
                    cv=3
                ),
                n_jobs=-1
            )
    )
])

print("Treinando...")
pipeline.fit(X_train2, Y_train2)
print("Treino concluído.")

MODEL_PATH = "modelo_traits.joblib"
LABELS_PATH = "traits_labels.json"

joblib.dump(pipeline, MODEL_PATH)
with open(LABELS_PATH, "w", encoding="utf-8") as f:
    json.dump(mlb2.classes_.tolist(), f, ensure_ascii=False, indent=2)

print("Modelo salvo em:", MODEL_PATH)
print("Labels salvos em:", LABELS_PATH)
print("Tamanho treino:", len(X_train), "tamanho teste:", len(X_test))

len(X_texts2): 11536
len(Y_list2): 11536
Y2.shape: (11536, 50)
Labels (n): 50
Shapes consistentes. Y2.shape: (11536, 50)
Treinando...


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['agil', 'cioso', 'determinada', 'honesto', 'humilde', 'irascível', 'modesto', 'paciente', 'sensível', 'sincero', 'sério'] will be ignored
  warnings.warn(


Treino concluído.
Modelo salvo em: modelo_traits.joblib
Labels salvos em: traits_labels.json
Tamanho treino: 9228 tamanho teste: 2308


In [ ]:
df_final.head()

,snippet,traits,trait_list,snippet_clean
0,"O conservador, com um gesto nobre, rejeitou; e...","[""misterioso"", ""gentil"", ""inteligente"", ""pragm...","[misterioso, gentil, inteligente, pragmático, ...",conservador gesto nobre rejeitou rejeitei moci...
1,"--Sim, meu tio; tenho aqui. Accendi o charuto ...","[""conservador""]",[conservador],sim tio aqui accendi charuto baforei mar prime...
2,O Rio começava a apparecer-me. A victoria corr...,"[""vaidoso"", ""orgulhoso"", ""arrogante"", ""dominad...","[vaidoso, orgulhoso, arrogante, dominador]",rio comecava apparecer victoria corria cruzand...
3,"A victoria parou. Saltámos e eu, curioso de vê...","[""curioso"", ""observador""]","[curioso, observador]",victoria parou saltamos curioso ver admirar ma...
4,Os rumores deliciavam-me e enterneciam-me--dei...,"[""observador""]",[observador],rumores deliciavam enterneciam deixei tempo ou...


In [ ]:
probs = pipeline.predict_proba(X_test)  # shape: (n_samples, n_classes)

assert probs.shape[0] == Y_test.shape[0], f"n_samples mismatch: probs {probs.shape[0]} vs Y_test2 {Y_test.shape[0]}"
assert probs.shape[1] == Y_test.shape[1], f"n_classes mismatch: probs {probs.shape[1]} vs Y_test2 {Y_test.shape[1]}"

preds_05 = (probs >= 0.5).astype(int)
print("Hamming loss (0.5):", hamming_loss(Y_test2, preds_05))
print("Micro F1 (0.5):", f1_score(Y_test, preds_05, average="micro", zero_division=0))
print("Macro F1 (0.5):", f1_score(Y_test, preds_05, average="macro", zero_division=0))

best_thresh = []
for i in range(Y_test.shape[1]):
    y_true = Y_test[:, i]
    y_score = probs[:, i]
    # se não houver positivos verdadeiros, fallback para 0.5
    if y_true.sum() == 0:
        best_thresh.append(0.5)
        continue
    prec, rec, thr = precision_recall_curve(y_true, y_score)
    f1s = 2 * prec * rec / (prec + rec + 1e-12)
    if len(thr) == 0:
        best_thresh.append(0.5)
    else:
        best_thresh.append(float(thr[np.nanargmax(f1s)]))

best_thresh = np.array(best_thresh)
np.save(THRESH_PATH, best_thresh)
print("Thresholds salvos em:", THRESH_PATH)

preds_opt = (probs >= best_thresh[None, :]).astype(int)
print("Hamming loss (opt):", hamming_loss(Y_test2, preds_opt))
print("Micro F1 (opt):", f1_score(Y_test, preds_opt, average="micro", zero_division=0))
print("Macro F1 (opt):", f1_score(Y_test, preds_opt, average="macro", zero_division=0))

Hamming loss (0.5): 0.010641247833622183
Micro F1 (0.5): 0.7456503728251864
Macro F1 (0.5): 0.7518192672068367
Thresholds salvos em: data/traits_thresholds.npy
Hamming loss (opt): 0.013336221837088389
Micro F1 (opt): 0.740253164556962
Macro F1 (opt): 0.7982333589065864


In [ ]:
report = classification_report(Y_test2, preds_opt, target_names=mlb.classes_, zero_division=0)
print(report)

               precision    recall  f1-score   support

       astuto       0.77      0.69      0.73        59
  inteligente       0.42      0.53      0.47        85
   misterioso       0.72      0.73      0.72        66
    impulsivo       0.96      0.60      0.74        72
  determinado       0.28      1.00      0.44        91
     corajoso       0.96      0.82      0.88        61
         leal       0.83      0.69      0.75        62
   observador       0.24      1.00      0.38        82
     protetor       0.73      0.70      0.72        70
      curioso       0.67      0.64      0.65        58
    arrogante       0.91      0.70      0.79        56
    orgulhoso       0.59      0.71      0.65        70
       cínico       0.98      0.81      0.88        57
      vaidoso       0.82      0.67      0.74        55
    ambicioso       1.00      0.62      0.77        64
      teimoso       0.93      0.73      0.82        56
    dominador       0.90      0.69      0.78        55
  control

In [ ]:
def predict_traits_snippet(snippet, max_k=2):
    probs = pipeline_2.predict_proba([snippet])[0]

    try:
        thresh = np.load(THRESH_PATH)
        if len(thresh) != len(mlb.classes_):
            thresh = np.array([0.5] * len(mlb.classes_))
    except:
        thresh = np.array([0.5] * len(mlb.classes_))


    selected = [(lab, float(p)) for lab, p, t in zip(mlb.classes_, probs, thresh) if p >= t]
    if len(selected) > max_k:
        selected = sorted(selected, key=lambda x: x[1], reverse=True)[:max_k]

    if len(selected) == 0:
        all_sorted = sorted(zip(mlb.classes_, probs), key=lambda x: x[1], reverse=True)
        selected = [(lab, float(p)) for lab, p in all_sorted[:max_k]]

    return {
        "labels": [lab for lab, p in selected],
        "scores": {lab: p for lab, p in selected}
    }

# teste rápido
ex = X_test[30]
print("Snippet exemplo:", ex)
print("Predição:", predict_traits_snippet(ex))


Snippet exemplo: Na sala de visitas a menina Adelaide collocava por suas proprias mãos
heras e flores em volta das serpentinas.
Predição: {'labels': ['trabalhador', 'responsável'], 'scores': {'trabalhador': 0.6205194928192599, 'responsável': 0.6090902213674513}}


In [ ]:
variation_dict = {
    "miguel": [
        "Miguel sempre se destacava pela maneira como observava o mundo antes de se mover. "
        "Nada nele era precipitado; tudo parecia seguir um ritmo interno silencioso, disciplinado, quase científico. "
        "Costumava escutar as pessoas com atenção real, anotando mentalmente detalhes que a maioria ignorava: "
        "contradições, hesitações, receios escondidos. No entanto, sua racionalidade nunca significou frieza absoluta. "
        "Pelo contrário — Miguel carregava uma espécie de responsabilidade moral que o obrigava a agir quando percebia "
        "injustiças, principalmente se envolviam alguém mais vulnerável. Era protetor por impulso ético, não por vaidade. "
        "Quando surgia um conflito, sua postura sempre combinava calma e firmeza. Ele podia até estar tremendo por dentro, "
        "mas por fora mantinha o controle, movido pela convicção de que decisões tomadas no calor do medo quase sempre "
        "feriam mais do que resolviam. O grupo confiava nele porque Miguel nunca fugia das responsabilidades difíceis. "
        "Às vezes ele parecia distante — não por arrogância, mas porque carregava sozinho o peso de pensar três passos "
        "à frente. Sua coragem não era barulhenta: era feita de escolhas difíceis, assumidas com serenidade e lealdade "
        "absoluta aos amigos."
    ],

    "cranio": [
        "Crânio tinha a mente inquieta de quem enxerga padrões onde outros só veem confusão. Bastava um detalhe, uma pista, "
        "um lampejo para que ele começasse a conectar possibilidades, testar hipóteses e, quase sempre, chegar a conclusões "
        "brilhantes. Nunca fazia isso em silêncio: explicava o raciocínio com prazer, como se resolver problemas fosse sua "
        "forma de brincar com o mundo. Às vezes sua inteligência vinha acompanhada de uma pitada de arrogância — um levantar "
        "de sobrancelha, um comentário sarcástico — mas quem o conhecia sabia que aquilo escondia um entusiasmo quase infantil "
        "com descobertas. Seus olhos sempre estavam procurando algo: uma expressão que não bate, uma inconsistência na história, "
        "um número fora do lugar. Era metódico e inquieto, rígido com lógica e impaciente com imprecisões, mas extremamente "
        "generoso quando reconhecia o esforço de alguém. Costumava rir do próprio talento, não por ego, mas porque realmente "
        "se divertia com desafios intelectuais. E, mesmo sendo brilhante, Crânio tinha humildade suficiente para voltar atrás "
        "quando errava — recuperava-se rápido, resiliente, e voltava ao problema com renovada energia analítica."
    ],

    "chumbinho": [
        "Chumbinho sempre foi movido por uma coragem quase instintiva. Mesmo quando seu tamanho não ajudava, sua atitude "
        "preenchia a sala inteira. Enfrentava injustiças sem calcular o risco e, muitas vezes, colocava-se na frente dos "
        "outros sem pensar duas vezes. Mas sua impulsividade não o tornava inconsequente: conforme crescera, aprendera a "
        "transformar esse impulso inicial em ação estratégica. Sabia reunir informações, conversar com quem precisava, "
        "organizar testemunhas, planejar denúncias — sempre guiado por um senso de justiça forte e visceral. Ainda assim, "
        "havia nele uma sinceridade simples: Chumbinho não escondia medo quando o sentia. Confessava suas inseguranças, mas "
        "avançava mesmo assim, e era justamente esse tipo de coragem vulnerável que inspirava quem estava ao seu lado. Era "
        "leal até o fim e tinha uma desconfiança saudável que o fazia investigar além do óbvio. Quando estava prestes a agir, "
        "seus olhos brilhavam com determinação — aquele brilho típico de quem prefere arriscar do que ver alguém sofrer sem "
        "intervenção."
    ],

    "magrí": [
        "Magrí carregava no olhar uma mistura bonita de delicadeza e firmeza. Era disciplinada como poucos, dedicada aos "
        "treinos, às escolhas éticas, ao cuidado com os outros. Tinha um senso de justiça quase instintivo, capaz de fazê-la "
        "desistir de uma vitória quando percebia que alguém precisava de ajuda. Sua empatia não era ingênua: era consciente, "
        "equilibrada, guiada por valores sólidos que ela nunca abandonava, mesmo quando era mais fácil ceder à pressão. "
        "Apesar do temperamento calmo, Magrí não era passiva. Ela sabia confrontar injustiças com uma coragem tranquila, quase "
        "elegante, mantendo a serenidade mesmo diante de ataques ou provocações. Sua generosidade era prática, não "
        "performática; quando via alguém lutando, ela se aproximava sem alarde e oferecia apoio real. Em seus momentos mais "
        "desafiadores, Magrí respirava fundo, ajustava a postura e seguia adiante — não para ser admirada, mas porque "
        "acreditava profundamente no que era certo."
    ],

    "calu": [
        "Calu era energia pura. Sua curiosidade parecia infinita e sua imaginação nunca descansava. Era o tipo de pessoa que "
        "via potencial em qualquer situação: um código secreto, uma nova ideia, um plano ousado. Essa impulsividade criativa o "
        "levava tanto a soluções brilhantes quanto a pequenos desastres, mas Calu sempre se recuperava rápido. Para ele, errar "
        "nunca foi um problema — o verdadeiro fracasso era não tentar. Seu entusiasmo tinha uma força curiosa: unia o grupo, "
        "animava os amigos e transformava tensões em risos. Mas por trás da alegria constante havia uma certa insegurança "
        "silenciosa, aquela vontade de ser útil, de ajudar, de fazer parte de algo maior. Calu era generoso mesmo quando "
        "precisava aprender a se organizar melhor; era leal quando todos duvidavam; era a fagulha que acendia a motivação dos "
        "demais. Quando suas ideias davam certo, seu sorriso era sincero — não de orgulho, mas de celebração coletiva."
    ],

    "andrade": [
        "Andrade era um profissional que misturava rigor, paciência e humanidade em doses raras. Seu método era meticuloso: "
        "coletava depoimentos, organizava pistas, revisava documentos com olhar treinado e nunca tomava decisões sem checar "
        "cada detalhe. Porém, sua frieza técnica não anulava seu senso de compaixão; pelo contrário, ele fazia questão de "
        "proteger inocentes e sempre explicava suas decisões com clareza, sem arrogância. Coragem, para Andrade, não era "
        "gritar nem se exibir — era agir quando ninguém mais podia. Em campo, tomava decisões rápidas e seguras, mas no "
        "convívio com os jovens mostrava um lado mentor, paciente, capaz de ensinar com calma e firmeza. Sua ética "
        "profissional era tão sólida quanto sua disposição para admitir incertezas quando elas existiam. Andrade carregava nos "
        "ombros o peso da experiência, mas usava esse peso para abrir caminho para os outros, nunca para se impor."
    ]
}


for name, variations in variation_dict.items():
    print("\n======", name.upper(), "======")
    for t in variations:
        print("\nSnippet:", t)
        print("Pred:", predict_traits_snippet(t))




====== MIGUEL ======

Snippet: Miguel sempre se destacava pela maneira como observava o mundo antes de se mover. Nada nele era precipitado; tudo parecia seguir um ritmo interno silencioso, disciplinado, quase científico. Costumava escutar as pessoas com atenção real, anotando mentalmente detalhes que a maioria ignorava: contradições, hesitações, receios escondidos. No entanto, sua racionalidade nunca significou frieza absoluta. Pelo contrário — Miguel carregava uma espécie de responsabilidade moral que o obrigava a agir quando percebia injustiças, principalmente se envolviam alguém mais vulnerável. Era protetor por impulso ético, não por vaidade. Quando surgia um conflito, sua postura sempre combinava calma e firmeza. Ele podia até estar tremendo por dentro, mas por fora mantinha o controle, movido pela convicção de que decisões tomadas no calor do medo quase sempre feriam mais do que resolviam. O grupo confiava nele porque Miguel nunca fugia das responsabilidades difíceis. Às vezes e

trait_prompt = PromptTemplate(
    input_variables=["texto"],
    template=("""
        Você é um analista especializado em perfis de personagens literários. Com base EXCLUSIVAMENTE nos título de um dos livros disponíveis em domínio publico fornecidos abaixo, produza um JSON com inferências de personalidade. Regras obrigatórias:
        - A saída deve conter apenas JSON. Não inclua explicações.
        - O JSON final deve ter exatamente as chaves:
        - "inferences": lista de objetos.
        - "resume": um resumo com NO MÁXIMO 200 caracteres EM PORTUGUÊS BARSILEIRO sobre quem é o personagem com base EM TODOS OS TRECHOS enviado no contexto como "trechos_texto".
        - Cada objeto dentro de "inferences" deve seguir esta estrutura:
        - "id": o mesmo id recebido na entrada, correspondente ao trecho analisado.
        - "traits": lista de adjetivos que descrevem a personalidade detectada NO TRECHO.
        - Você deve retirar apenas traits claramente justificáveis pelo trecho.
        - Se não houver traits relevantes, NÃO inclua esse item na lista "inferences".
        - Todos os adjetivos devem ser:
        - em PORTUGUÊS BRASILEIRO,
        - no gênero masculino,
        - escolhidos APENAS desta lista:
         "astuto","inteligente","misterioso","impulsivo","determinado",
          "corajoso","leal","observador","protetor","curioso",
          "arrogante","orgulhoso","cínico","vaidoso","ambicioso",
          "teimoso","dominador","controlador","inflexível","desconfiado",
          "gentil","bondoso","generoso","altruísta","empático",
          "frio","calculista","rígido","prudente","precavido",
          "racional","sarcástico","irônico","questionador","criativo",
          "trabalhador","resiliente","responsável","conservador","pragmático",
          "agressivo","hostil","vingativo","cruel","mentiroso",
          "covarde","irresponsável","negligente","egoísta","maldoso",
        - Contexto: \n {texto}
    """),
)